In [1]:
import glob
import os
import h5py
import numpy as np
import scipy.io as sio
import pandas as pd
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Testing linear classifiers on top of PredNet representations

In [17]:
def get_features(feat_dir, layer_id, i):
    """
    get activations of a particular DNN layer for a particular image

    Input:
    feat_dir: Directory containing activations generated using generate_features.py
    layer_id: layer name
    i: image index

    Output:
    flattened activations
    """
    activations = glob.glob(feat_dir + "/*" + ".mat")
    activations.sort()
    feat = sio.loadmat(activations[i])[layer_id]
    return feat.ravel()

feat = get_features('./prednet/feats/92images/prednet_moments__representation__4_layers_epoch_1/', 'rep_all', 0)
feat.shape

(135936,)

In [29]:
labels = pd.read_csv('92_images.csv')

# feature_dir = './prednet/feats/92images/prednet_kitti_finetuned_moments__representation__10c/'
# feature_dir = './feats/92images_feats/alexnet/'
feature_dir = './prednet/feats/92images/prednet_moments__representation__4_layers_epoch_1/'
train_x = [get_features(feature_dir, 'rep_all', i) for i in range(len(labels))]
train_x = np.array(train_x)
train_y = np.array(labels.animate)

train_x.shape, train_y.shape

((92, 135936), (92,))

In [30]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#pca = PCA(n_components=45)
#x_pca = pca.fit(train_x).transform(train_x)

clf = svm.LinearSVC()
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
skf = StratifiedKFold(n_splits=4)
scores = cross_val_score(clf, train_x_scaled, train_y, cv=skf)
scores, scores.mean()

(array([0.54166667, 0.65217391, 0.39130435, 0.63636364]), 0.555377140974967)

In [91]:
len(labels[labels.animate == 1]) / len(labels)

0.5108695652173914

In [15]:
def evaluate(model_dir, data_x, data_y, model_type='linear'):
    
    # load model
    model_path = os.path.join(model_dir, model_type + '.pkl')
    model = joblib.load(model_path)
    
    scaler_path = os.path.join(model_dir, model_type + '_scaler.pkl')
    scaler = None
    if os.path.exists(scaler_path):
        scaler = joblib.load(scaler_path)
    
    preds = model.predict(data_x)
    acc = accuracy_score(data_y, preds)
    metric_str = 'acc: {}'.format(acc)
        
    print('Test {}'.format(metric_str))
    #f = open(os.path.join(results_dir, 'test.txt'), 'w')
    #f.write('Test results:\n')
    #f.write(metric_str)
    #f.close()
    
classifier_path = './classifier/models/linear__prednet_kitti_finetuned_moments_10c__2c_easy/'
evaluate(classifier_path, train_x, train_y)

Test acc: 0.510869565217
